In [0]:
%matplotlib inline

Classification by SVM of a 2D data set
===================

Define functions
----------------

In [0]:
import numpy as np
from matplotlib import pyplot as plt
# Visualization of the decision boundary and regions
def plot2d_classification(decision_function, X_train, y_train, X_test=None, y_test=None, w=None, cmap=plt.cm.bwr, xlim=None, ylim=None, levels=None, colors='k', linestyles=None):

    plt.figure()
    ax = plt.axes()

    if xlim is None:
        xlim = [X_train[:, 0].min() - .5, X_train[:, 0].max() + .5]
    if ylim is None:
        ylim = [X_train[:, 1].min() - .5, X_train[:, 1].max() + .5]

    xx, yy = np.meshgrid(np.arange(xlim[0], xlim[1], 0.02), np.arange(ylim[0], ylim[1], 0.02))    

    # Show prediction (P(y=+1 | X) by color by assigning a color to each point in the mesh [x_min, x_max]x[y_min, y_max].
    Z = decision_function(np.c_[xx.ravel(), yy.ravel()])
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    if levels is not None:
        ax.contour(xx, yy, Z, levels=levels, colors=colors, linestyles=linestyles, alpha=0.5)
    else:
        ax.pcolor(xx, yy, Z, cmap=cmap, alpha=0.1, edgecolors=None)

    # Plot the decision boundary
    if w is not None:
        x1 = np.linspace(xx.min(), xx.max(), 100)
        if w[2] != 0:
            x2 = -(w[0] + w[1] * x1) / w[2]
            cnd = np.logical_and(x2<yy.max(), x2>yy.min())
            plt.plot(x1[cnd], x2[cnd], 'k-')
        else:
            plt.axvline(x=-w[0]/w[1], color='k')

    # Plot also the training points
    ax.scatter(X_train[y_train>0, 0], X_train[y_train>0, 1], c='r',  marker='s', cmap=cmap, edgecolors='k', label='Training data', alpha=1)
    ax.scatter(X_train[y_train<=0, 0], X_train[y_train<=0, 1], c='b', marker='o', cmap=cmap, edgecolors='k', label='Training data', alpha=1)
    # and testing points if given
    if X_test is not None and y_test is not None:
        ax.scatter(X_test[y_test>0, 0], X_test[y_test>0, 1], c='r',  marker='s', cmap=cmap, edgecolors='k', label='Training data', alpha=1)
        ax.scatter(X_test[y_test<=0, 0], X_test[y_test<=0, 1], c='b', marker='o', cmap=cmap, edgecolors='k', label='Training data', alpha=1)
        plt.legend(loc="upper right", fontsize=16, frameon=True)
        ax.get_legend().legendHandles[0].set_color('k')
        ax.get_legend().legendHandles[1].set_color('k')

    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    plt.axis('tight')
    plt.xlabel('x1', fontsize=16)
    plt.ylabel('x2', fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.gca().set_aspect('equal')
    plt.tight_layout()

In [0]:
def  histogram_predict(decision_function, X_train, y_train, X_test=None, y_test=None, bins=None, normed=False):
    if bins is None:
        bins = len(y_train) // 4
    plt.figure()
    ax = plt.axes()
    pred = decision_function(X_train)
    plt.hist( [ pred[y_train>0], pred[y_train<=0] ], bins=bins, histtype='stepfilled', normed=normed, alpha=0.5, color=['r', 'b'], label=['$y=+1$', '$y=-1$'])
    if X_test is not None and y_test is not None:
        pred = decision_function(X_test)
        plt.hist( [ pred[y_test>0], pred[y_test<=0] ], bins=bins, histtype='stepfilled', normed=normed, alpha=0.3, color=['r', 'b'], label=['$y_{test}=+1$', '$y_{test}=-1$'])
    plt.xlabel("$g(x)$", fontsize=16)
    plt.ylabel("Frequency", fontsize=16)
    plt.legend(loc="upper right", fontsize=16, frameon=True)
    plt.axis('tight')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    from matplotlib.ticker import FormatStrFormatter
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%1.0f'))
    plt.tight_layout()

Make training data
------------------

In [0]:
# Example 1: define manually
X = np.array([[0, 0], [1,0], [0,1], [1,1]])
y = np.array([-1,1,1,1])

In [0]:
# Example 2: draw npos and nneg points from the Gaussian distribution for each class
npos = 30
nneg = 30
np.random.seed(321)
X = np.r_[np.random.randn(npos, 2) + [3, 3], np.random.randn(nneg, 2)]
# [1,1,...,1,-1,-1,...,-1]
y = np.array([1] * npos + [-1] * nneg)

In [0]:
# Example 3: create moons using sklearn
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.2, random_state=0)
y[y==0] = -1

In [0]:
# Example 4: create circles using sklearn
from sklearn.datasets import make_circles
X, y = make_circles(n_samples=150, noise=0.1, random_state=0, factor=0.3)
y[y==0] = -1

Plot the training points

In [0]:
# Plot the training points
ax = plt.figure()
ax = plt.axes()
ax.scatter(X[y>0, 0], X[y>0, 1], c='r',  marker='s', cmap=plt.cm.bwr, edgecolors='k', label='Training data', alpha=1)
ax.scatter(X[y<=0, 0], X[y<=0, 1], c='b', marker='o', cmap=plt.cm.bwr, edgecolors='k', label='Training data', alpha=1)
plt.xlabel('x1', fontsize=16)
plt.ylabel('x2', fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.gca().set_aspect('equal')
ax.set_xlim(X[:,0].min()-0.5, X[:,0].max()+0.5)
ax.set_ylim(X[:,1].min()-0.5, X[:,1].max()+0.5)
plt.tight_layout()

Run the training
----------------

In [0]:
# Linear SVM
from sklearn.svm import SVC

model = SVC(kernel='linear', C=100)
model.fit(X,y)

w = np.c_[model.intercept_, model.coef_].ravel()
print(w)

In [0]:
# Visualize the decision boundary and margin
plot2d_classification(lambda X: model.decision_function(X), X, y, w=w, levels=[-1,0,1], linestyles=['--', '-', '--'])
plt.savefig('linear_svm.png', transparent=True)
histogram_predict(lambda X: model.decision_function(X), X, y)
plt.savefig('hist_linear_svm.png', transparent=True)

In [0]:
# Nonlinear SVM
model = SVC(kernel='rbf', C=1)                     # try C=0.1, 1, 10, 100, etc
#model = SVC(kernel='poly', degree=5, C=100)  # try degree=1, 2, 3, ..., 10, etc
model.fit(X,y)

In [0]:
# Visualize the decision boundary and margin
plot2d_classification(lambda X: model.decision_function(X), X, y, levels=[-1,0,1], linestyles=['--', '-', '--'])
histogram_predict(lambda X: model.decision_function(X), X, y)